In [5]:
pip install olefile pdfplumber

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import zlib
import olefile
import pdfplumber
import struct

# ==========================================
# 1. 파일들이 저장된 폴더 이름
SOURCE_DIR = "this"
# ==========================================

def get_hwp_text(filename):
    # HWP 파일 열기
    f = olefile.OleFileIO(filename)
    dirs = f.listdir()

    # HWP 파일 본문 찾기
    sections = [d for d in dirs if d[0] == "BodyText"]
    text = ""

    for section in sections:
        bodytext = f.openstream(section)
        data = bodytext.read()
        
        # HWP의 텍스트는 압축되어 있음
        unpacked_data = zlib.decompress(data, -15)
        
        # 텍스트 추출
        i = 0
        size = len(unpacked_data)
        while i < size:
            header = struct.unpack_from("<I", unpacked_data, i)[0]
            rec_type = header & 0x3ff
            rec_len = (header >> 20) & 0xfff

            if rec_type == 67: # 텍스트 레코드
                rec_data = unpacked_data[i+4:i+4+rec_len]
                # 한글/영어 텍스트만 디코딩
                text += rec_data.decode('utf-16-le')
                text += "\n"

            i += 4 + rec_len

    return text

# ==========================================
# 2. 변환 작업 시작
# ==========================================
if not os.path.exists(SOURCE_DIR):
    print(f"'{SOURCE_DIR}' 폴더가 없습니다.")
    exit()

print(f">>> '{SOURCE_DIR}' 폴더의 파일을 변환합니다.")

files = os.listdir(SOURCE_DIR)
total_files = len(files)
count = 0

for idx, filename in enumerate(files):
    file_path = os.path.join(SOURCE_DIR, filename)
    filename_lower = filename.lower()
    
    # 텍스트 파일 저장 경로 (.txt)
    txt_filename = os.path.splitext(filename)[0] + ".txt"
    txt_path = os.path.join(SOURCE_DIR, txt_filename)
    
    # 이미 변환했으면 패스
    if os.path.exists(txt_path):
        continue

    try:
        # 1. HWP 파일 변환
        if filename_lower.endswith(".hwp"):
            print(f"[{idx+1}/{total_files}] HWP 변환: {filename}")
            
            # 텍스트 추출
            extracted_text = get_hwp_text(file_path)
            
            # 저장
            with open(txt_path, "w", encoding="utf-8") as f:
                f.write(extracted_text)
            count += 1

        # 2. PDF 파일 변환
        elif filename_lower.endswith(".pdf"):
            print(f"[{idx+1}/{total_files}] PDF 변환: {filename}")
            
            text_content = []
            with pdfplumber.open(file_path) as pdf:
                for page in pdf.pages:
                    extracted = page.extract_text()
                    if extracted:
                        text_content.append(extracted)
            
            if text_content:
                full_text = "\n".join(text_content)
                with open(txt_path, "w", encoding="utf-8") as f:
                    f.write(full_text)
                count += 1
            else:
                print(f"그림으로 된 PDF : 글자 추출 불가.")

    except Exception as e:
        print(f"   실패 ({filename}): {e}")
        # 실패한 파일 삭제
        if os.path.exists(txt_path):
            os.remove(txt_path)

print(f"\n>>> 작업 완료! 총 {count}개의 파일을 텍스트로 바꿨습니다.")

>>> 📂 'this' 폴더의 파일을 변환합니다... (크롬 안 씀)
[1/550] 🔄 HWP 변환: 금융통화위원회 의사록(2003년도 제16차)(2003.8.7).hwp
   실패 (금융통화위원회 의사록(2003년도 제16차)(2003.8.7).hwp): not an OLE2 structured storage file
[2/550] 🔄 HWP 변환: 금융통화위원회 의사록(2002년도 제3차)(2002.1.17).hwp
   실패 (금융통화위원회 의사록(2002년도 제3차)(2002.1.17).hwp): not an OLE2 structured storage file
[3/550] 🔄 PDF 변환: 금융통화위원회 의사록(2015년도 제8차)(2015.4.23.).pdf
[4/550] 🔄 PDF 변환: 금융통화위원회 의사록(2021년도 제11차)(2021.6.10)_1.pdf
[5/550] 🔄 HWP 변환: 금융통화위원회 의사록(2003년도 제8차)(2003.4.10).hwp
   실패 (금융통화위원회 의사록(2003년도 제8차)(2003.4.10).hwp): not an OLE2 structured storage file
[6/550] 🔄 PDF 변환: 금융통화위원회 의사록(2007년도 제2차)(2007.1.11).pdf
[7/550] 🔄 PDF 변환: 금융통화위원회 의사록(2012년도 제17차)(2012.9.13.).pdf
[8/550] 🔄 PDF 변환: 금융통화위원회 의사록(2014년도 제1차)(2014.1.9).pdf
[9/550] 🔄 PDF 변환: 금융통화위원회 의사록(2016년도 제12차)(2016.6.23.).pdf
[10/550] 🔄 PDF 변환: 금융통화위원회 의사록(2019년도 제16차)(2019.8.30.).pdf
[11/550] 🔄 PDF 변환: 금융통화위원회 의사록(2010년도 제7차)(2010.3.25).pdf
[12/550] 🔄 HWP 변환: 금융통화위원회 의사록(2004년도 제20차)(2004.9.23).hwp
[13/550] 🔄 P

In [9]:
import os

# ==========================================
# 1. 파일이 들어있는 폴더 이름
SOURCE_DIR = "this" 
# ==========================================

if not os.path.exists(SOURCE_DIR):
    print(f"!!! '{SOURCE_DIR}' 폴더가 없습니다. 이름을 확인해주세요.")
    exit()

# 폴더 안의 모든 파일 가져오기
all_files = os.listdir(SOURCE_DIR)

# 원본 파일(hwp, pdf)과 텍스트 파일(txt) 분류하기
source_files = [f for f in all_files if f.lower().endswith(('.hwp', '.pdf'))]
txt_files = [os.path.splitext(f)[0] for f in all_files if f.lower().endswith('.txt')]

fail_list = []

print(f">>> '{SOURCE_DIR}' 폴더에서 실패한 파일을 찾는 중...\n")

for file in source_files:
    # 확장자를 뺀 파일 이름
    name_without_ext = os.path.splitext(file)[0]
    
    # 만약 원본 파일 이름과 똑같은 .txt 파일이 없다면? -> 실패한 것!
    if name_without_ext not in txt_files:
        fail_list.append(file)

# 결과 출력
if fail_list:
    print(f"변환에 실패한 파일이 총 {len(fail_list)}개 발견되었습니다:")
    print("-" * 50)
    for i, fail_file in enumerate(fail_list):
        print(f"[{i+1}] {fail_file}")
    print("-" * 50)
    
    # 메모장 파일로 저장하기
    with open("변환_실패_목록.txt", "w", encoding="utf-8") as f:
        f.write(f"=== 변환 실패 목록 (총 {len(fail_list)}개) ===\n")
        for fail_file in fail_list:
            f.write(fail_file + "\n")
    print(f"\n'변환_실패_목록.txt' 파일로 저장되었습니다. 확인해보세요!")
else:
    print("실패한 파일 없이 모두 변환되었습니다.")

>>> 'this' 폴더에서 실패한 파일을 찾는 중...

변환에 실패한 파일이 총 104개 발견되었습니다:
--------------------------------------------------
[1] 금융통화위원회 의사록(2003년도 제16차)(2003.8.7).hwp
[2] 금융통화위원회 의사록(2002년도 제3차)(2002.1.17).hwp
[3] 금융통화위원회 의사록(2003년도 제8차)(2003.4.10).hwp
[4] 금융통화위원회 의사록(2002년도 제23차)(2002.9.26).hwp
[5] 금융통화위원회 의사록(2000년도 제5차)(2000.2.10).hwp
[6] 금융통화위원회 의사록(2001년도 제2차)(2001.1.11).hwp
[7] 금융통화위원회 의사록(2003년도 제15차)(2003.7.24).hwp
[8] 금융통화위원회 의사록(2003년도 제1차)(2003.1.9).hwp
[9] 금융통화위원회 의사록(2000년도 제11차)(2000.3.30).hwp
[10] 금융통화위원회 의사록(1999년도 제16차)(1999.5.6).hwp
[11] 금융통화위원회 의사록(2000년도 제12차)(2000.4.6).hwp
[12] 금융통화위원회 의사록(2001년도 제24차)(2001.9.6).hwp
[13] 금융통화위원회 의사록(1999년도 제13차)(1999.4.8).hwp
[14] 금융통화위원회 의사록(2001년도 제21차)(2001.8.9).hwp
[15] 금융통화위원회 의사록(1999년도 제10차)(1999.3.4).hwp
[16] 금융통화위원회 의사록(2002년도 제10차)(2002.4.4).hwp
[17] 금융통화위원회 의사록(1999년도 제22차)(1999.7.15).hwp
[18] 금융통화위원회의사록(1999년도제36차)(1999.12.2).hwp
[19] 금융통화위원회 의사록(2003년도 제7차)(2003.3.27).hwp
[20] 금융통화위원회 의사록(2000년도 제25차)(2000.9.21).hwp
[21] 금융통화위원회 의

In [10]:
import os

# ==========================================
# 1. 파일이 들어있는 폴더 이름
SOURCE_DIR = "this" 
# ==========================================

if not os.path.exists(SOURCE_DIR):
    print(f"!!! '{SOURCE_DIR}' 폴더를 찾을 수 없음")
    exit()

# 폴더 안의 모든 파일 목록 가져오기
all_files = os.listdir(SOURCE_DIR)

# 2. 파일 분류하기
# 원본 파일 (PDF + HWP)
origin_files = [f for f in all_files if f.lower().endswith(('.pdf', '.hwp'))]
# 텍스트 파일 (TXT)
txt_files = [f for f in all_files if f.lower().endswith('.txt')]

# 3. 이름만 따서 비교하기 (확장자 떼고 이름만)
origin_names = set(os.path.splitext(f)[0] for f in origin_files)
txt_names = set(os.path.splitext(f)[0] for f in txt_files)

# 4. 결과 출력 (초간단 버전)
print(f"📊 [파일 개수 확인 결과]")
print(f"------------------------------------")
print(f"원본 파일(PDF+HWP) : {len(origin_files)}개")
print(f"변환된 텍스트(TXT)  : {len(txt_files)}개")
print(f"------------------------------------")

if len(origin_files) == len(txt_files):
    print("모든 파일이 완벽하게 텍스트로 바뀜!")
else:
    diff_count = len(origin_files) - len(txt_files)
    print(f"{diff_count}개의 파일이 아직 텍스트로 안 바뀜!")
    
    # 안 바뀐거 찾기
    missing = origin_names - txt_names
    if missing:
        print("\n[아직 텍스트 파일이 없는 원본 목록]")
        for i, name in enumerate(missing):
            # 실제 원본 파일명을 찾아서 보여줌
            real_file = [f for f in origin_files if f.startswith(name)][0]
            print(f"   {i+1}. {real_file}")

print(f"------------------------------------")

📊 [파일 개수 확인 결과]
------------------------------------
원본 파일(PDF+HWP) : 438개
변환된 텍스트(TXT)  : 438개
------------------------------------
모든 파일이 완벽하게 텍스트로 바뀜!
------------------------------------


In [ ]:
import os
import re

# ==========================================
# 1. 폴더 이름 확인
SOURCE_DIR = "this" 
# ==========================================

if not os.path.exists(SOURCE_DIR):
    print(f"'{SOURCE_DIR}' 폴더가 없어요.")
    exit()

print(f">>> 남은 파일들도 날짜 형식으로 변환을 시작합니다...\n")

files = os.listdir(SOURCE_DIR)
count = 0

for filename in files:
    if not filename.lower().endswith(".txt"):
        continue
    
    # 이미 8자리 숫자로만 된 파일은 건너뛰기
    if re.match(r'^\d{8}\.txt$', filename):
        continue

    # 패턴 1: (2023. 10. 13) 또는 (2023.10.13) 또는 2023.10.13. 등 모든 조합
    match = re.search(r'(\d{4})[\.\s]*(\d{1,2})[\.\s]*(\d{1,2})', filename)
    
    if match:
        year, month, day = match.groups()
        # 8자리 숫자로 조립
        new_date_name = f"{year}{month.zfill(2)}{day.zfill(2)}.txt"
        
        old_path = os.path.join(SOURCE_DIR, filename)
        new_path = os.path.join(SOURCE_DIR, new_date_name)

        # 중복 파일 처리 (_1, _2 붙이기)
        if os.path.exists(new_path) and filename != new_date_name:
            base = f"{year}{month.zfill(2)}{day.zfill(2)}"
            uniq = 1
            while os.path.exists(os.path.join(SOURCE_DIR, f"{base}_{uniq}.txt")):
                uniq += 1
            new_path = os.path.join(SOURCE_DIR, f"{base}_{uniq}.txt")
            new_date_name = f"{base}_{uniq}.txt"

        try:
            os.rename(old_path, new_path)
            print(f"변경 성공: {filename}  ->  {new_date_name}")
            count += 1
        except Exception as e:
            print(f"변경 실패: {filename} ({e})")

print(f"\n>>> 🏁 작업 끝! 추가로 {count}개의 이름을 바꿨습니다.")

>>> 🔍 남은 파일들도 날짜 형식으로 변환을 시작합니다...

✨ 변경 성공: 금융통화위원회 의사록(2022년도 제10차)(2022.5.26)_1.txt  ->  20220526.txt
✨ 변경 성공: 금융통화위원회 의사록(2006년도 제26차)(2006.12.21).txt  ->  20061221.txt
✨ 변경 성공: 금융통화위원회 의사록(2025년도 제5차)(2025.3.13).txt  ->  20250313.txt
✨ 변경 성공: 금융통화위원회 의사록(2014년도 제19차)(2014.10.15.).txt  ->  20141015.txt
✨ 변경 성공: 금융통화위원회 의사록(2019년도 제10차)(2019.5.31.).txt  ->  20190531.txt
✨ 변경 성공: 금융통화위원회 의사록(2021년도 제10차)(2021.5.27)_1.txt  ->  20210527.txt
✨ 변경 성공: 금융통화위원회 의사록(2010년도 제9차)(2010.4.22).txt  ->  20100422.txt
✨ 변경 성공: 금융통화위원회 의사록(2013년도 제5차)(2013.3.14).txt  ->  20130314.txt
✨ 변경 성공: 금융통화위원회 의사록(2015년도 제11차)(2015.6.11.).txt  ->  20150611.txt
✨ 변경 성공: 금융통화위원회 의사록(2004년도 제19차)(2004.9.9).txt  ->  20040909.txt
✨ 변경 성공: 금융통화위원회 의사록(2013년도 제24차)(2013.12.26.).txt  ->  20131226.txt
✨ 변경 성공: 금융통화위원회 의사록(2009년도 제20차)(2009.9.10).txt  ->  20090910.txt
✨ 변경 성공: 금융통화위원회 의사록(2011년도 제28차)(2011.12.16).txt  ->  20111216.txt
✨ 변경 성공: 금융통화위원회 의사록(2014년도 제11차)(2014.6.12.).txt  ->  20140612.txt
✨ 변경 성공: 금융통화위원회 